In [1]:
import numpy as np
import pandas as pd
from scipy.special import logsumexp

## Scenario
Let's build a scenario. We have an imbalanced dataset with sock color and scariness.

- Individuals with Blue socks have a 1/30=0.033 chance of being violent.
- Individuals with Red socks have a 9/70=0.129 chance of being violent.

However, individuals with Red socks make up a 0.7 fraction of our total population.

We would like to privatize this data to protect individual's privacy. Let's see what happens to our different populations.

NOTE: I demonstrate the weirdness of the exponential mechanism at high epsilons, but I validated that our understanding of it aligns with existing (and reliable) implementations online. I show it anyway, but please pay more attention to the example at the bottom : )

In [2]:
# Select paradigm
group_name = 'sock_color'
group_marker = ['Blue', 'Red']
attribute_name = 'scary'
attribute = ['Violent', 'NonViolent'] # Rare attribute first

In [3]:
def build_data(b_sock_count=30, r_sock_count=70, b_val_count=1, r_val_count=9):
    socks = [group_marker[0]] * b_sock_count + [group_marker[1]] * r_sock_count
    valedictiorians = ([attribute[0]] * b_val_count + [attribute[1]] * (b_sock_count-b_val_count)) + \
                      ([attribute[0]] * r_val_count + [attribute[1]] * (r_sock_count-r_val_count))
    df = pd.DataFrame({group_name: socks, attribute_name: valedictiorians})
    return df

In [4]:
df = build_data()
df

,sock_color,scary
0,Blue,Violent
1,Blue,NonViolent
2,Blue,NonViolent
3,Blue,NonViolent
4,Blue,NonViolent
...,...,...
95,Red,NonViolent
96,Red,NonViolent
97,Red,NonViolent
98,Red,NonViolent


In [5]:
options = df['scary'].unique()

def score(data, option, print_score=False):
    return data.value_counts()[option]/len(data)

print(score(df['scary'], 'Violent'))
print(score(df['scary'], 'NonViolent'))

0.1
0.9


In [6]:
def exponential(x, R, u, sensitivity, epsilon, print_scores=False, return_probs=False):
    # Score for each R
    scores = [u(x, r) for r in R]
    if print_scores:
        print(scores)
    
    # Prob of each element, based on R
    probabilities = [np.exp((epsilon * score) / (2 * sensitivity)) for score in scores]
    
    # Probabilities need to be normalized
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

    # Element from r using the probabilities
    if print_scores:
        print(probabilities)
    if return_probs:
        return probabilities

    return np.random.choice(R, 1, p=probabilities)[0]

exponential(df['scary'], options, score, 1, 5.5, print_scores=True)

[0.1, 0.9]
[0.09975049 0.90024951]


'NonViolent'

## Making sure that my implementation of the exponential mechanism is reasonable
Grabbed a different implementation from the web. The odd results at high epsilons persist. Oh well!

In [7]:
r = [exponential(df['scary'], options, score, 1, 1.0) for i in range(100)]
pd.Series(r).value_counts()

NonViolent    51
Violent       49
dtype: int64

In [8]:
r = [exponential(df['scary'], options, score, 1, 10) for i in range(100)]
pd.Series(r).value_counts()

NonViolent    97
Violent        3
dtype: int64

In [9]:
def exponential_mechanism(q, eps, sensitivity, prng=np.random, monotonic=False):
    coef = 1.0 if monotonic else 0.5
    scores = coef*eps/sensitivity*q
    probas = np.exp(scores - logsumexp(scores))
    return prng.choice(q.size, p=probas)

In [10]:
q = [score(df['scary'], r) for r in options]

In [11]:
r = [options[exponential_mechanism(np.array(q), 1, 1.0)] for i in range(100)]
pd.Series(r).value_counts()

NonViolent    62
Violent       38
dtype: int64

In [12]:
r = [options[exponential_mechanism(np.array(q), 10, 1.0)] for i in range(100)]
pd.Series(r).value_counts()

NonViolent    98
Violent        2
dtype: int64

## Let's ignore the odd behavior of the exponential mechanism for now.
### Let's ignore the odd behavior for now. Let's instead focus in on the probability of reporting valedictorian correctly conditioned on group membership, at a fixed epsilon value of 1.0.

In [13]:
options
# x, R, u, sensitivity, epsilon, print_scores=False

array(['Violent', 'NonViolent'], dtype=object)

In [14]:
alpha = len(df[df['sock_color'] == 'Red'])/len(df)
epsilon = 10.0
probs = exponential(df[df['sock_color'] == 'Red']['scary'], options, score, 1, alpha*epsilon, print_scores=True, return_probs=True)
print(list(zip(options,probs)))

[0.12857142857142856, 0.8714285714285714]
[0.06913842 0.93086158]
[('Violent', 0.06913842034334683), ('NonViolent', 0.9308615796566532)]


In [16]:
inv_alpha = 1 - alpha
probs = exponential(df[df['sock_color'] == 'Blue']['scary'], options, score, 1, inv_alpha*epsilon, print_scores=True, return_probs=True)
print(list(zip(options,probs)))

[0.03333333333333333, 0.9666666666666667]
[0.19781611 0.80218389]
[('Violent', 0.19781611144141822), ('NonViolent', 0.8021838885585817)]


### Something pernicious is going on here.
Recall that Blue socks have a 1/30=0.033 chance of being violent, while Red socks have a 9/70=0.129 chance of being violent.

However, at epsilon=1, we find that, due to the group imbalance, we **overvalue** the potential of Blue socks to be violent (significantly!) while we **undervalue** Red socks' potential. 

*The probability of producing the majority value for the minority class is lower than the probability for the majority class (!)*

*The probability of producing the minority value for the minority class is higher than the probability for the majority class (!)*